In [7]:
from sys import argv

In [ ]:
path = argv[1]

import pandas as pd
df = pd.read_json(path)[['body_markdown','tags']]
df.info()

In [5]:
new = df.head(10)

In [1]:
import pandas as pd
df = pd.read_json('1414384\MathStackExchangeAPI_Part_1_TimeStamps_1512760268_1535031491.json\MathStackExchangeAPI_Part_1_TimeStamps_1512760268_1535031491.json')[['answers','tags']]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121700 entries, 0 to 121699
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   answers  90892 non-null   object
 1   tags     121700 non-null  object
dtypes: object(2)
memory usage: 1.9+ MB


In [2]:
df.head(10)

,answers,tags
0,NaN,"[inequality, convex-analysis, norm, projection]"
1,NaN,"[representation-theory, lie-groups, lie-algebras]"
2,NaN,[multivariable-calculus]
3,NaN,"[partial-derivative, jacobian]"
4,"[{'community_owned_date': 1535031083, 'answer_...","[sequences-and-series, logic, set-theory, ordi..."
5,NaN,[game-theory]
6,NaN,[linear-algebra]
7,"[{'answer_id': 2892122, 'body_markdown': 'Hint...","[integration, analysis]"
8,NaN,"[p-adic-number-theory, nonarchimedian-analysis]"
9,NaN,"[integration, definite-integrals, special-func..."


In [7]:
# удаляем все вопросы у которых нет ответов (нулевые ответы)
df = df[~df['answers'].isnull()]

In [11]:
df

,answers,tags
4,"[{'community_owned_date': 1535031083, 'answer_...","[sequences-and-series, logic, set-theory, ordi..."
7,"[{'answer_id': 2892122, 'body_markdown': 'Hint...","[integration, analysis]"
11,"[{'answer_id': 2892113, 'body_markdown': 'As I...","[linear-algebra, functions]"
12,"[{'answer_id': 2892117, 'body_markdown': ' I ...",[complex-analysis]
23,"[{'answer_id': 2892082, 'body_markdown': 'It i...","[calculus, differential-equations, derivatives]"
...,...,...
121695,"[{'answer_id': 2557628, 'body_markdown': '*Not...","[set-theory, forcing]"
121696,"[{'answer_id': 2557604, 'body_markdown': 'What...",[finite-fields]
121697,"[{'answer_id': 2557652, 'body_markdown': 'Let&...",[polynomials]
121698,"[{'answer_id': 2557602, 'body_markdown': 'Hint...","[real-analysis, measure-theory, lebesgue-integ..."


In [20]:
# восстанавливаем правильную индексацию
df=df.reset_index()

In [ ]:
# собираем это в один 'плоский' датафрейм
rows = []
i=list(df['tags'].keys())[0]
progress = 0

for answers in df['answers']:
    if answers is not None:
        for answer in answers:
            row = {}
            markdown = answer['body_markdown']
            row['markdown']=markdown
            row['tags']=df['tags'][i]
            rows.append(row)
    i+=1 
    progress+=1
    print(progress/len(df))

df_rows = pd.DataFrame.from_dict(rows)

In [33]:
print(df_rows)

                                                 markdown  \
0       They just mean that the sequence has indices $...   
1       Hint:\r\n\begin{align}\r\n\ln  \left( \prod_{j...   
2       As I mentioned in my comment, [the definition ...   
3       \r\nI think you can not apply the identity the...   
4       The maximum modulus principle states that a ho...   
...                                                   ...   
147065  If you want to simplify the calculations you c...   
147066  Let&#39;s say the parameter $d$ is real.  Then...   
147067  Hint: If $m(E) &gt; 0,$ then $M(\chi_E)(x) &gt...   
147068        Hint: $$(xyx^{-1})^2=(xyx^{-1})(xyx^{-1})$$   
147069  Base case: when $n=1$, we have of course $$(xy...   

                                                     tags  
0       [sequences-and-series, logic, set-theory, ordi...  
1                                 [integration, analysis]  
2                             [linear-algebra, functions]  
3                          

In [26]:
# на входе у нас уже не джейсон, а датафрейм df_rows
# из предыдущей ячейки

import re
import json



def extract_substrings(text): 
    # Находим фразы, начинающиеся и заканчивающиеся знаками доллара или начинающиеся и заканчивающиеся /begin{} и /end{} 
    matches = re.findall(r'(\$\$.+?\$\$|\$.+?\$|/begin\{(.*?)\}(.*?)/end\{\2\})', text) 
    return [match[0] if match[0] else f'/begin{{{match[1]}}}{match[2]}/end{{{match[1]}}}' for match in matches] 
    # разделить поэтапно

def process_json_file(input_json):
#     # Загружаем данные из JSON-файла
#     with open(file_path, 'r', encoding='utf-8') as json_file:
#         input_json = json.load(json_file)

    # Извлекаем нужные данные из JSON
    data = {'body_markdown': input_json['markdown']}

    # Преобразуем данные в DataFrame, добавляя индекс
    df = pd.DataFrame(data)

    # Создаем новый столбец для хранения вычлененных подстрок
    df['formulae'] = df['body_markdown'].apply(extract_substrings)
    df['tags'] = input_json['tags']

    # Создаем новый DataFrame для хранения вычлененных подстрок
    extracted_phrases_df = pd.DataFrame({'extracted_phrases': [item for sublist in df['formulae'].tolist() for item in sublist]})

    # Заменяем подстроки на [[i]] в исходной таблице
    for index, row in df.iterrows():
        for i, substring in enumerate(row['formulae']):
            df.at[index, 'body_markdown'] = df.at[index, 'body_markdown'].replace(substring, f'[[{i}]]')

    # Возвращаем результат
    return df, extracted_phrases_df

# Пример использования
#json_file_path = '100_questions.json'
result_dataframe, extracted_phrases_dataframe = process_json_file(df_rows)

In [84]:
print(result_dataframe)


                                            body_markdown  \
0       They just mean that the sequence has indices [...   
1       Hint:\r\n\begin{align}\r\n\ln  \left( \prod_{j...   
2       As I mentioned in my comment, [the definition ...   
3       \r\nI think you can not apply the identity the...   
4       The maximum modulus principle states that a ho...   
...                                                   ...   
147065  If you want to simplify the calculations you c...   
147066  Let&#39;s say the parameter [[0]] is real.  Th...   
147067         Hint: If [[0]] then [[1]] for every [[2]]    
147068  Hint: [[0]](xyx^{-1})^2=(xyx^{-1})(xyx^{-1})[[0]]   
147069  Base case: when [[0]], we have of course [[1]]...   

                                                 formulae  \
0       [$1, 2, 3, \ldots$, $\omega$, $A$, $\omega$, $...   
1                                                      []   
2       [$1$, $f\colon \mathbb{R}^{m} \to \mathbb{R}^{...   
3       [$S$, $f=g$, $D

In [34]:
extracted_phrases_dataframe

,extracted_phrases
0,"$1, 2, 3, \ldots$"
1,$\omega$
2,$A$
3,$\omega$
4,$A$
...,...
1652143,$$
1652144,$$
1652145,$n=k+1$
1652146,$$


In [ ]:
text = ""
i=0
for phrase in extracted_phrases_dataframe['extracted_phrases']:
    if phrase!="$$" and len(phrase)>3:
        text += phrase
        print(f"phrase: {phrase}")
    elif phrase == "$$": 
        print('empty')
    elif len(phrase)<=3:
        print('too short')
    print(f"progress = {i/1652148}")

In [44]:
import nltk
nltk.download('punkt')

allWords = nltk.tokenize.word_tokenize(text)
allWordDist = nltk.FreqDist(allWords)
allWordDist.most_common(10)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\blody\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


[('$', 1848992),
 ('{', 483068),
 ('}', 483006),
 (')', 393361),
 ('(', 391370),
 (',', 198702),
 (';', 126767),
 ('=', 125391),
 ('x', 116375),
 ('&', 111960)]

In [50]:
allWordDist.most_common(500)

[('$', 1848992),
 ('{', 483068),
 ('}', 483006),
 (')', 393361),
 ('(', 391370),
 (',', 198702),
 (';', 126767),
 ('=', 125391),
 ('x', 116375),
 ('&', 111960),
 ('1', 71530),
 ('0', 70603),
 ('f', 58549),
 ('\\', 53400),
 ('\\mathbb', 46136),
 ('n', 45483),
 ('2', 42839),
 ('\\frac', 42478),
 ('[', 39883),
 ('#', 39206),
 (']', 38157),
 ('39', 37073),
 ('a', 35658),
 ('+', 35654),
 ('lt', 34952),
 ('y', 34071),
 ('\\in', 32318),
 ('gt', 30782),
 ('R', 29576),
 ('A', 28549),
 ('b', 24490),
 (':', 24149),
 ('-1', 23998),
 ('-', 23029),
 ('t', 21699),
 ('X', 20693),
 ('B', 19049),
 ('3', 18314),
 ('^', 18076),
 ('\\to', 17551),
 ('*', 17278),
 ('P', 17114),
 ('g', 16865),
 ('p', 16795),
 ('k', 16356),
 ('N', 16317),
 ('C', 15440),
 ('z', 15279),
 ('Z', 14428),
 ('\\sqrt', 14368),
 ('F', 12485),
 ('\\mathcal', 12437),
 ('c', 12268),
 ('i', 12051),
 ('\\infty', 11579),
 ('\\alpha', 11553),
 ('m', 10872),
 ('4', 10706),
 ('r', 10217),
 ('v', 10100),
 ('d', 10076),
 ('u', 9949),
 ('\\Bbb', 9

In [80]:
com_freq = {}
for com in allWordDist.most_common():
    if com[1]>=500 and len(com[0])>2 and com[0][0]=="\\":
        com_freq[com[0]] = com[1]
        

In [81]:
# команды которые были замечены в использовании больше 500 раз
com_freq

{'\\mathbb': 46136,
 '\\frac': 42478,
 '\\in': 32318,
 '\\to': 17551,
 '\\sqrt': 14368,
 '\\mathcal': 12437,
 '\\infty': 11579,
 '\\alpha': 11553,
 '\\Bbb': 9735,
 '\\mathbf': 9422,
 '\\pi': 8905,
 '\\times': 8408,
 '\\le': 8118,
 '\\text': 7786,
 '\\phi': 7768,
 '\\epsilon': 7640,
 '\\leq': 7366,
 '\\cdot': 7326,
 '\\lambda': 6979,
 '\\theta': 6915,
 '\\dfrac': 6697,
 '\\right': 6346,
 '\\left': 6086,
 '\\cap': 5946,
 '\\ldots': 5944,
 '\\delta': 5860,
 '\\sin': 5774,
 '\\partial': 5424,
 '\\dots': 5324,
 '\\sum_': 5263,
 '\\ge': 5008,
 '\\beta': 4947,
 '\\equiv': 4932,
 '\\geq': 4898,
 '\\vec': 4894,
 '\\overline': 4804,
 '\\cos': 4720,
 '\\sigma': 4608,
 '\\mu': 4607,
 '\\operatorname': 4539,
 '\\mathrm': 4504,
 '\\rightarrow': 4240,
 '\\lim_': 4228,
 '\\neq': 4042,
 '\\subset': 3897,
 '\\langle': 3760,
 '\\forall': 3746,
 '\\tag': 3711,
 '\\subseteq': 3686,
 '\\varphi': 3645,
 '\\binom': 3490,
 '\\omega': 3419,
 '\\implies': 3381,
 '\\gamma': 3381,
 '\\log': 3338,
 '\\bar': 3286,
 

In [140]:
with open("most_common.txt", "w",encoding="utf-8") as text_file:
    text_file.write(str(allWordDist.most_common()))

In [82]:
len(com_dsfreq.keys())

159

In [106]:
# далее - списки из наиболее часто употребляемых команд из статьи 
# https://blog.writefull.com/the-100-most-frequent-latex-commands/
# и из com_freq

# команды из статьи
freq_list_from_paper = ['\\frac', '\\right', '\\left','\\alpha','\\mu',
                        '\\pi','\\sigma','\\sum','\\lambda','\\beta',
                       '\\nu','\\delta','\\Delta','\\rho','\\phi',
                        '\\Phi','\\gamma','\\Gamma','\\omega','\\Omega',
                        '\\over','\\bar','\\sqrt','\\theta','\\Theta',
                        '\\tau','\\rangle','\\hat','\\tilde','\\cal',
                        '\\psi','\\vec','\\langle','\\epsilon','\\varepsilon',
                        '\\cdot','\\in','\\xi','\\Xi','\\infty',
                        '\\mathcal','\\times','\\prime','\\pm','\\be',
                        '\\ee','\\chi','\\ell','\\text','\\to',
                        '\\varphi','\\leq','\\cos','\\overline','\\sin',
                        '\\kappa','\\rightarrow','\\Righarrow','\\dagger','\\otimes',
                        '\\equiv','\\zeta','\\dot','\\ln']

# команды из ответов, употреблявшиеся больше 500 раз
freq_list_from_data = ['\\frac','\\in','\\to','\\sqrt','\\mathcal',
                       '\\infty','\\alpha','\\pi','\\times','\\le',
                       '\\text','\\phi','\\epsilon','\\leq','\\cdot',
                       '\\lambda','\\theta','\\dfrac','\\right','\\left',
                       '\\cap','\\ldots','\\delta','\\sin','\\partial',
                       '\\dots','\\sum','\\ge','\\beta','\\equiv',
                       '\\geq','\\vec','\\overline','\\cos','\\sigma',
                       '\\mu','\\operatorname','\\mathrm','\\rightarrow','\\lim',
                       '\\neq','\\subset','\\langle','\\forall','\\tag',
                       '\\subseteq','\\varphi','\\binom','\\omega','\\implies',
                       '\\gamma','\\log','\\bar','\\ln','\\cup',
                       '\\varepsilon','\\Omega','\\cdots','\\pm','\\mid',
                       '\\circ','\\ne','\\mathfrak','\hat','\\bmatrix',
                       '\\Delta','\\psi','\\tau','\\int','\\rangle',
                       '\\mapsto','\\mod']



In [131]:
# находим пересечения -- слова, которые есть и в первом и во втором списках
# всего 43 пересечения
overall_frequents = set(freq_list_from_paper).intersection(freq_list_from_data)
print(f"Количество пересечений: {len(overall_frequents)}")

# отсортировыываем в порядке убывания частоты использования, основываясь на данных, 
# полученных из вопросов
sorted_frequents = sorted(overall_frequents, key=lambda x: com_freq[x],reverse=True)

# итоговый список слов, к которым необходимо сделать словарь
print(sorted_frequents)

Количество пересечений: 43
['\\frac', '\\in', '\\to', '\\sqrt', '\\mathcal', '\\infty', '\\alpha', '\\pi', '\\times', '\\text', '\\phi', '\\epsilon', '\\leq', '\\cdot', '\\lambda', '\\theta', '\\right', '\\left', '\\delta', '\\sin', '\\beta', '\\equiv', '\\vec', '\\overline', '\\cos', '\\sigma', '\\mu', '\\rightarrow', '\\langle', '\\varphi', '\\omega', '\\gamma', '\\bar', '\\ln', '\\varepsilon', '\\Omega', '\\pm', '\\hat', '\\Delta', '\\psi', '\\tau', '\\rangle', '\\sum']


In [ ]:
# некоторые команды не являются напрямую математическими операциями ИЛИ являются надстрочными символами
# или ещё каким-либо образом не заменяются простой заменой
# т.ч. в первой итерации не рассматривались
# это были команды: \frac, \mathcal, \text, \left и \right, \vec, \overline, \langle, \rangle, \bar, \hat 

# т.ж. была удалена команда: \phi --> varphi (чтобы мы приводили \phi и 'фи' к \varphi)
# удалена \epsilon --> добавлена к \varepsilon: 'эпсилон', '\epsilon'
# \rightarrow и \to -- один и тот же символ, т.ч. предположим что \to проще писать и будем приводить к нему
# т.е. удалим \rightarrow из словаря

# к некоторым греческим буквам были добавлены их заглавные эквиваленты

#  некоторые альтернативы для дальнейших имплементаций
# '\\frac' : ['дробь от а делить на б', 'а делить на б']
# '\\to': ['\rightarrow','следовательно', 'откуда следует', 'отображение из а в б', 
#                              'влечёт', 'отсюда следует', 'если а то б', 'стремится к'],
#  '\\sum': ['сумма', 'сумма от а до б']}

# от себя т.ж. были добавлены '!', '^' и '_' (командами не являются, но необходимы)
# при дальнейшем рассмотрении были найдены в списке наиболее частых символов
# ('^', 18076)
# ('!', 9246)
# ('_', 8401)


replacement_dict = { '\\in': ['принадлежит', 'входит в', 'из'],
                     '\\to': ['\\rightarrow','следовательно', 'откуда следует', 'влечёт', 'отсюда следует', 'стремится к'],
                     '\\sqrt': ['корень из', 'корень квадратный из', 'квадратный корень из'],
                     '\\infty': ['бесконечность'],
                     '\\alpha': ['альфа'],
                     '\\pi': ['пи'],
                     '\\times': ['умножить на', 'произведение'],
                     '\\varphi': ['фи','\\phi'],
                     '\\leq': ['меньше или равно', '\\leqslant'],
                     '\\cdot': ['умножить на', 'скалярное произведение', 'внутреннее произведение'],
                     '\\Lambda': ['лямбда большая', 'лямбда заглавная'],
                     '\\lambda': ['лямбда', 'лямбда малая'],
                     '\\Theta': ['тета большая', 'тета заглавная'],
                     '\\theta': ['тета', 'тета малая'],
                     '\\Delta': ['дельта большая', 'дельта заглавная'],
                     '\\delta': [ 'дельта','дельта малая'],
                     '\\sin': ['синус'],
                     '\\beta': ['бета'],
                     '\\equiv': ['эквивалентно', 'тождественно равно', 'тождественно равняется'],
                     '\\cos': ['косинус'],
                     '\\Sigma': ['сигма'], 
                     '\\sigma':['сигма малая'],
                     '\\mu': ['мю'],
                     '\\Omega': ['омега заглавная', 'омега большая'],
                     '\\omega': ['омега','омега малая'],
                     '\\gamma': ['гамма','гамма малая'],
                     '\\ln': ['натуральный логарифм','логарифм по основанию e' ,'логарифм'],
                     '\\varepsilon': ['\\epsilon','эпсилон'],
                     '\\pm': ['плюс минус', 'плюс-минус'],
                     '\\psi': ['пси', 'пси малая'],
                     '\\Psi': ['пси большая', 'пси заглавная'],
                     '\\tau': ['тау', 'тау малая'],
                     '\\sum': ['сумма'],
                     '^' : ['в степени'],
                     '_' : ['с индексом'],
                     '!': ['факториал'] }

In [139]:
# итого, длина начального словаря замен - 36 слов. 
len(replacement_dict)


36

In [177]:
# выглядит не очень но. допустим

final = [(text, form, tags) for (text, form, tags) in zip(result_dataframe['body_markdown'],
                                                        result_dataframe['formulae'],
                                                        result_dataframe['tags']) if len(form) and '$$' not in form]

In [178]:
len(final)

62254

In [165]:
len(result_dataframe)

147070

In [167]:
fin_df = pd.DataFrame.from_records(final, columns =['body_markdown', 'formulae', 'tags'])

In [179]:
fin_df

,body_markdown,formulae,tags
0,They just mean that the sequence has indices [...,"[$1, 2, 3, \ldots$, $\omega$, $A$, $\omega$, $...","[sequences-and-series, logic, set-theory, ordi..."
1,\r\nI think you can not apply the identity the...,"[$S$, $f=g$, $D$, $D$, $S=\partial \mathbb D$,...",[complex-analysis]
2,The maximum modulus principle states that a ho...,"[$h(z)=f(z)-g(z)$, $\partial\Bbb D$, $0\leq|h(...",[complex-analysis]
3,"When you rearrange the differential equation, ...","[$y&#39;$, $0$, $x$, $y&#39;$]","[calculus, differential-equations, derivatives]"
4,I&#39;ll try to give some more details as an a...,"[$\,P^2=P$, $\;P(I-P)=0$, $\operatorname{Im}(I...","[linear-algebra, diagonalization]"
...,...,...,...
62249,"*Not quite an answer, but too long for a comme...","[$V$, $L$, $\neg CH$, $\Pi_1$, $\Sigma_1$, $\n...","[set-theory, forcing]"
62250,What I would do:\r\n\r\n- The polynomial [[0]]...,"[$x^4+x^3+1$, $2^4=16$, $15$, $x+x^2$, $3$, $5...",[finite-fields]
62251,If you want to simplify the calculations you c...,"[$ax^3+bx^2+cx+d$, $a,b,c,d\in \{0,1\}$, $15$,...",[finite-fields]
62252,Let&#39;s say the parameter [[0]] is real. Th...,"[$d$, $\text{Im}((X+iY)^4 + a (X+iY)^3 +b (X+i...",[polynomials]
